# Analyze eye signal data

created: November 19, 2024 <br>
last modified:  February 11, 2024

Start by analyzing a downsampled version of eye data, then introduce full resolution eye data and compare resting state aperiodic activity in eyes open vs close and in specific timepoint when eyes closed become open (or viceversa)

In [ ]:
# Imports
import neo
import numpy as np
import pandas as pd
import nixio
import quantities as pq
import matplotlib.pyplot as plt

# custom
import sys
sys.path.append("../../code")
from paths import EXTERNAL_PATH

In [2]:
# set path
eye_path = EXTERNAL_PATH + "/V1_v4_1024_electrode_resting_state_data/data/L_RS_090817/eye_signals/"

In [3]:
# Load data
with neo.NixIO(eye_path + "aligned_eye_data.nix", mode='ro') as nio:
    block = nio.read_block()

block

Block with 1 segments
name: 'Eye signals'
description: 'eye position and diameters'
annotations: {'nix_name': 'neo.block.4296992d060d4ee7b15db59fadc15740'}
file_datetime: datetime.datetime(2021, 4, 19, 13, 5, 17, 913450)
rec_datetime: datetime.datetime(2021, 4, 19, 6, 25, 32)
# segments (N=1)
0: Segment with 4 analogsignals, 1 epochs
   name: 'eye signal segment'
   description: 'Segment of eye pos and diam'
   annotations: {'nix_name': 'neo.segment.c532f30c3a734eabae2418c49282dada'}
   # analogsignals (N=4)
   0: AnalogSignal with 1 channels of length 39627730; units mV; datatype int16 
      name: 'XPos'
      annotations: {'nix_name': 'neo.analogsignal.19582fa74cef491f8a907698b6ad6702'}
      sampling rate: 30000.0 Hz
      time: 0.0 s to 1320.9243333333334 s
   1: AnalogSignal with 1 channels of length 39627730; units mV; datatype int16 
      name: 'YPos'
      annotations: {'nix_name': 'neo.analogsignal.62564a856339447fa99eb204ee0daddc'}
      sampling rate: 30000.0 Hz
      time

In [4]:
# set datetime to None to avoid errors when saving
# Neo Github issue: https://github.com/NeuralEnsemble/python-neo/issues/1198

print(f"Original datetime: {block.rec_datetime}")

# set block and segment datetime to None
block.rec_datetime = None
for segment in block.segments:
    segment.rec_datetime = None

print(f"New datetime: {block.rec_datetime}")


Original datetime: 2021-04-19 06:25:32
New datetime: None


In [5]:
# with neo.io.NixIO(eye_path + "L_RS_090817_aligned_eye_data.nix", mode='ow') as nio:
#     nio.write_block(block)

# print(f"Block saved to {eye_path}L_RS_090817_aligned_eye_data.nix")

In [9]:
# [0] for xpos and [1] for ypos
xdiam = block.segments[0].analogsignals[2] / 1000
ydiam = block.segments[0].analogsignals[3] / 1000
xdiam[xdiam < 0] = 0*pq.mV
ydiam[ydiam < 0] = 0*pq.mV

# we use .magnitude to access values
diam = np.sqrt(ydiam.magnitude**2 + xdiam.magnitude**2)
print(f"pupil diameter for every time point: {diam.shape}")

pupil diameter for every time point: (39627730, 1)


In [10]:
### Estimate behavioural epochs
# Empirically estimated thresholds for eye closure, according to Chen et al
#   if 'L_RS_090817' in eye_path: thr = 0.101

mask = (diam > 0.101)
behavioural_state = mask.astype(float)[:, 0]
behaviour_anasig = neo.core.AnalogSignal(behavioural_state,
                                              units=pq.V,
                                              sampling_rate=xdiam.sampling_rate,
                                              name='Behavioural state')
block.segments[0].analogsignals.append(behaviour_anasig)

### Smoothen states with sliding window
w = 3
kernel = [1/w]*w
behavioural_state = np.convolve(behavioural_state, kernel, mode='same')
behavioural_state[behavioural_state < 0.5] = 0
behavioural_state[behavioural_state >= 0.5] = 1

In [11]:
unique, counts = np.unique(behavioural_state, return_counts=True)
print(f"Eyes open or closed measured in binary: {unique} \neyes closed: {counts[0]} \neyes open:{counts[1]}")

Eyes open or closed measured in binary: [0. 1.] 
eyes closed: 32473667 
eyes open:7154063


In [19]:
# create behavioural state df
eyes = pd.DataFrame()
eyes['state'] = behavioural_state
eyes['behavioural_state'] = list(map(lambda x: 'open' if x == 1 else 'closed', behavioural_state))
eyes['diam'] = diam

# test if behavioural labels have been applied correctly
eyes[eyes['behavioural_state'] == 'closed']

,state,behavioural_state,diam
87947,0.0,closed,0.0
87948,0.0,closed,0.0
87949,0.0,closed,0.0
87950,0.0,closed,0.0
87951,0.0,closed,0.0
...,...,...,...
39592814,0.0,closed,0.0
39592815,0.0,closed,0.0
39592816,0.0,closed,0.0
39592817,0.0,closed,0.0


In [20]:
# define function for when creating the epoch object
def get_mean_state(diam):
    if np.sum(diam <= 0.5) > np.sum(diam > 0.5):
        state = 'Closed_eyes'
    else:
        state = 'Open_eyes'
    return state

In [21]:
## create epoch object
wh = np.where(np.diff(behavioural_state) != 0)[0]
edgeindex = [0] + wh.tolist() + [len(behavioural_state)]

# initialise with first slice
i_start = [edgeindex[0]]
i_stop = [edgeindex[1]]
states = [get_mean_state(behavioural_state[edgeindex[0]:edgeindex[1]])]
# Loop over indices, assign states and merge consecutive same-state slices
for startidx, stopidx in zip(edgeindex[1:-1], edgeindex[2:]):
    nextstate = get_mean_state(behavioural_state[startidx:stopidx])
    if nextstate == states[-1]:
        i_stop[-1] = stopidx
    else:
        i_start.append(startidx)
        i_stop.append(stopidx)
        states.append(nextstate)

# Turn index lists into time arrays
start_times = (np.array(i_start) / ydiam.sampling_rate).rescale('s')
stop_times = (np.array(i_stop) / ydiam.sampling_rate).rescale('s')
durs = stop_times - start_times

# Convert into a pandas dataframe,
datadict = {'t_start': start_times.magnitude,
                't_stop': stop_times.magnitude,
                'dur': durs.magnitude,
                'state': states}

epochs = pd.DataFrame(data=datadict)

In [22]:
epochs

,t_start,t_stop,dur,state
0,0.000000,2.931533,2.931533,Open_eyes
1,2.931533,4.317033,1.385500,Closed_eyes
2,4.317033,7.983133,3.666100,Open_eyes
3,7.983133,8.172400,0.189267,Closed_eyes
4,8.172400,8.430667,0.258267,Open_eyes
...,...,...,...,...
1098,1318.439600,1318.448233,0.008633,Open_eyes
1099,1318.448233,1319.579900,1.131667,Closed_eyes
1100,1319.579900,1319.743367,0.163467,Open_eyes
1101,1319.743367,1319.760600,0.017233,Closed_eyes


In [26]:
# confirm that we have 5 analogsignals (our 4 original + behaviour)
block.segments[0]

Segment with 5 analogsignals, 1 epochs
name: 'eye signal segment'
description: 'Segment of eye pos and diam'
annotations: {'nix_name': 'neo.segment.c532f30c3a734eabae2418c49282dada'}
# analogsignals (N=5)
0: AnalogSignal with 1 channels of length 39627730; units mV; datatype int16 
   name: 'XPos'
   annotations: {'nix_name': 'neo.analogsignal.19582fa74cef491f8a907698b6ad6702'}
   sampling rate: 30000.0 Hz
   time: 0.0 s to 1320.9243333333334 s
1: AnalogSignal with 1 channels of length 39627730; units mV; datatype int16 
   name: 'YPos'
   annotations: {'nix_name': 'neo.analogsignal.62564a856339447fa99eb204ee0daddc'}
   sampling rate: 30000.0 Hz
   time: 0.0 s to 1320.9243333333334 s
2: AnalogSignal with 1 channels of length 39627730; units mV; datatype int16 
   name: 'XDiam'
   annotations: {'nix_name': 'neo.analogsignal.6ec6601136fb4b00be8f79bbf5e4ad78'}
   sampling rate: 30000.0 Hz
   time: 0.0 s to 1320.9243333333334 s
3: AnalogSignal with 1 channels of length 39627730; units mV; 

In [23]:
# save file 
with neo.io.NixIO(eye_path + "L_RS_090817_aligned_eye_data.nix", mode='ow') as nio:
    nio.write_block(block)

print(f"Block saved to {eye_path}L_RS_090817_aligned_eye_data.nix")

Block saved to E:/V1_v4_1024_electrode_resting_state_data/data/L_RS_090817/eye_signals/L_RS_090817_aligned_eye_data.nix
